## Example HXMM01: test the MS-simulation modules

This demo is used to illustrate the capability of produce referecen model images from model properties
#### input file: demo_disk3d.inp


In [1]:
import sys
import glob
import os
import io
import logging
import emcee
import pprint as pp
#from pprint import pprint

print(sys.version)

import socket 
if  'hypersion' or 'mini' in socket.gethostname() :
    os.chdir('/Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/output/')
print(socket.gethostname())
print(os.getcwd())

import gmake
pp.pprint(gmake.__version__)
pp.pprint(gmake.__email__)
pp.pprint(gmake.__demo__)
gmake.check_setup()

#pp.pprint(gmake.meta.pars_def,indent=4,width=100)
#gmake.pprint(gmake.meta.pars_def,indent=4,width=100)
#gmake.pprint(gmake.meta.xymodel_header,indent=4,width=100)

inpfile=gmake.__demo__+'/../examples/inpfile/hxmm01_b6c3_uv_mc.inp'
logfile=''

print('>'*40)
gmake.logger_config()
gmake.logger_status()
#gmake.logger_status(root=True)

outdir='/Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/output/hxmm01_b6c3_uv_mc/'
gmake.logger_config(logfile=outdir+'/gmake.log',loglevel='DEBUG',logfilelevel='DEBUG')
gmake.logger_status()

print('\n'*5)
inp_dct=gmake.read_inp(inpfile)
gmake.pprint(inp_dct,indent=4,width=100)

print()
#def_dct=gmake.read_inp('/Users/Rui/Dropbox/Worklib/projects/GMaKE/gmake/metadata/parameter_definition.inp') 
#print("="*80)
#pprint(def_dct)



3.7.5 (default, Oct 19 2019, 11:15:26) 
[Clang 11.0.0 (clang-1100.0.33.8)]
hyperion
/Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/output
**********exe read_inp()**************


Python version:   3.7.5 (default, Oct 19 2019, 11:15:26) 
[Clang 11.0.0 (clang-1100.0.33.8)]
Host Name:        hyperion
Num of Core:      8
Total Memory:     32.0 GB
Available Memory: 11.42 GB
################################################################################
astropy            >=3.2.2      3.2.2       
emcee              >=3.0.0      3.0.0       
corner             >=2.0        2.0.1       
tqdm               unspecified  4.31.1      
lmfit              unspecified  0.9.12      
asteval            >=0.9.14     0.9.14      
numexpr            >=2.7.0      2.7.0       
hickle             unspecified  3.4.5       
alpy               unspecified  0.22.0      
regions            unspecified  0.5.dev1001 
scipy              unspecified  1.2.1       
reproject          unspecified  0.6.dev646  
python-casacore    >=3.1.1      3.1.1       
scikit-image       unspecified  0.14.2      
galpy              unspecified  1.5.dev0    
mkl-fft            unspecified  1.0.14      
pvextr

'0.2.dev1'
'rx.astro@gmail.com'
'/Users/Rui/Dropbox/Worklib/projects/GMaKE/gmake'
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<Logger gmake (DEBUG)>
[<StreamHandler stderr (INFO)>]
<Logger gmake (DEBUG)>
[<FileHandler /Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/output/hxmm01_b6c3_uv_mc/gmake.log (DEBUG)>, <StreamHandler stderr (DEBUG)>]






**********exe read_inp()**************
{   'hxmm01-basics': {'object': 'hxmm01', 'z': 2.308},
    'co76': {   'restfreq': <Quantity 806.65181 GHz>,
                'ncrit': <Quantity 120000. 1 / cm3>,
                'type': 'disk3d',
                'vis': '../data/hxmm01/alma/2015.1.00723.S/bb34.ms'},
    'ci21': {   'restfreq': <Quantity 809.34197 GHz>,
                'ncrit': <Quantity 1300. 1 / cm3>,
                'type': 'disk3d',
                'vis': '../data/hxmm01/alma/2015.1.00723.S/bb34.ms'},
    'h2o': {   'restfreq': <Quantity 752.03314 GHz>,
               'ncrit': <Quantity 21000000. 1 / cm3>,
               'type': 'disk3d

In [2]:
dat_dct=gmake.read_data(inp_dct,fill_mask=True,fill_error=True,save_data=True)
mod_dct=gmake.inp2mod(inp_dct)
#gmake.pprint(mod_dct)


read data (may take some time..)

Read: ../data/hxmm01/alma/2015.1.00723.S/bb34.ms

data@../data/hxmm01/alma/2015.1.00723.S/bb34.ms              (118553, 161)        146 MiB             
uvw@../data/hxmm01/alma/2015.1.00723.S/bb34.ms               (118553, 3)            1 MiB             
weight@../data/hxmm01/alma/2015.1.00723.S/bb34.ms            (118553,)            463 KiB             1505.6934
chanfreq@../data/hxmm01/alma/2015.1.00723.S/bb34.ms          (161,)       242.8624 GHz   245.3623 GHz
chanwidth@../data/hxmm01/alma/2015.1.00723.S/bb34.ms         (161,)        15.6241 MHz    15.6241 MHz
phasecenter@../data/hxmm01/alma/2015.1.00723.S/bb34.ms       2h20m16.613s  -6d01m43.15s
data flagging fraction: 0.008205361199930865

Read: ../data/hxmm01/alma/2015.1.00723.S/bb1.ms

data@../data/hxmm01/alma/2015.1.00723.S/bb1.ms               (118430, 110)         99 MiB             
uvw@../data/hxmm01/alma/2015.1.00723.S/bb1.ms                (118430, 3)            1 MiB             
weigh

In [9]:
from gmake import model_lnprob
fit_dct,sampler=gmake.fit_setup(inp_dct,dat_dct)
#lnl,blobs=model_lnprob(fit_dct['p_start'],fit_dct,inp_dct,dat_dct,
#                               savemodel=inp_dct['general']['outdir'],packblobs=True)
pp.pprint(fit_dct)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
optimizer: emcee
optimizing parameters: index / name / start / lo_limit / up_limit / scale
 0   xypos.ra@compa        35.0693875    (  35.0688319  ,   35.0699431   )   0.0000056   
 1   xypos.dec@compa       -6.0283111    (  -6.0288667  ,   -6.0277556   )   0.0000056   
 2   vsys@compa              292.00      (    -120.00   ,     280.00     )     4.12      
 3   vrot[1:5]@compa         500.00      (     0.00     ,     800.00     )     5.00      
 4   vdis[0:5]@compa         120.00      (     0.00     ,     800.00     )     6.80      
 5   pa@compa                -14.00      (    -94.00    ,      66.00     )     0.80      
 6   inc@compa                79.00      (     5.00     ,      85.00     )     0.74      
 7   lineflux@co76-compa      1.30       (     0.10     ,     200.00     )     1.99      
 8   sbser[0]@co76-compa      0.22       (     0.01     ,      1.00      )     0.01      
 9   lin

('o', <Quantity [-2.  ,  2.  ,  0.15] arcsec>)
--> xypos.ra@compa deg 35.0693875
('o', <Quantity [-2.  ,  2.  ,  0.15] arcsec>)
--> xypos.dec@compa deg -6.028311111111111
('a', <Quantity [-120.,  280.,   40.] km / s>)
--> vsys@compa km/s 292.0
('a', <Quantity [  0., 800.,  40.] km / s>)
--> vrot[1:5]@compa km/s [500. 500. 500. 500.]
('a', <Quantity [  0., 800.,  10.] km / s>)
--> vdis[0:5]@compa km/s [120. 120. 120. 120. 120.]
('o', <Quantity [-80.,  80.,   5.] deg>)
--> pa@compa deg -14.0
('a', <Quantity [ 5., 85.,  5.] deg>)
--> inc@compa deg 79.0
('a', <Quantity [1.e-01, 2.e+02, 5.e-02] Jy km / s>)
--> lineflux@co76-compa Jy*km/s 1.3
('a', <Quantity [0.01, 1.  , 0.01] arcsec>)
--> sbser[0]@co76-compa arcsec 0.21709
('a', <Quantity [1.e-01, 2.e+02, 1.e-02] Jy km / s>)
--> lineflux@ci21-compa Jy*km/s 0.65
('a', <Quantity [0.01, 1.  , 0.01] arcsec>)
--> sbser[0]@ci21-compa arcsec 0.18771
('a', <Quantity [1.e-01, 2.e+02, 1.e-02] Jy km / s>)
--> lineflux@h2o-compa Jy*km/s 0.38
('a', <Qua

In [ ]:
from gmake.metadata import template_imheader

In [ ]:
pprint(template_imheader)

In [ ]:
import gmake

In [ ]:
gmake.meta.pars_def